In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
CP = pd.read_csv('CP_ontology_pvalue_data_Mitotox_Mechanisms_658_scaled.csv') 
CP

,viability_apoptosis_vb_percent_dead_only_Chromatin Modifiers,viability_apoptosis_vb_percent_dead_only_ER Stress/UPR,viability_apoptosis_vb_percent_dead_only_Apoptosis,viability_apoptosis_vb_percent_dead_only_RTK,viability_apoptosis_vb_percent_dead_only_Cytoskeletal Re-org/Integrin,viability_apoptosis_vb_percent_dead_only_Spindle Dynamics,viability_apoptosis_vb_percent_dead_only_Known cancer gene,viability_apoptosis_vb_percent_dead_only_RNA Polymerase II,viability_death_vb_num_live_cells_Metabolism,viability_death_vb_num_live_cells_WNT,...,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_Metabolism,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_WNT,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_ROS,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_Cytoskeletal Re-org/Integrin,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_JAK/STAT,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_Spindle Dynamics,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_RAS,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_Known cancer gene,cell_cycle_cell_cycle_count_cc_g1_plus_g2_count_RNA Polymerase II,InChICode_standardised
0,0.856109,0.956310,0.661301,0.935187,-1.688069,-1.454033,-1.362097,-1.345309,0.822402,0.833629,...,0.966776,1.001054,0.956249,0.812115,-1.513231,0.869083,0.818542,0.893099,0.924417,"InChI=1S/C32H41NO2/c1-31(2,3)26-18-16-25(17-19..."
1,-1.728855,0.956310,-2.264476,0.194311,-1.688069,0.720890,0.792188,0.781404,-1.410468,0.833629,...,0.966776,-0.873795,-0.138232,0.812115,0.803496,-1.166329,0.818542,0.893099,-1.094787,InChI=1S/C10H10N2O/c1-8-7-10(13)12(11-8)9-5-3-...
2,0.604550,-1.535441,-0.053351,-1.656308,0.701175,0.720890,0.792188,0.781404,0.822402,0.833629,...,0.966776,1.001054,0.956249,-1.254582,-1.261181,0.869083,-1.481265,-1.129686,0.924417,"InChI=1S/C10H10N4O2S/c11-8-2-4-9(5-3-8)17(15,1..."
3,0.856109,-1.535441,0.661301,0.935187,0.701175,0.720890,0.792188,0.781404,-0.984302,-0.756872,...,0.790698,-1.034894,-1.240543,-1.254582,0.206534,0.869083,-1.243617,0.893099,-1.094787,"InChI=1S/C10H11ClO3/c1-10(2,9(12)13)14-8-5-3-7..."
4,-0.495514,-0.138102,-1.731964,-0.691095,-1.688069,-1.454033,-1.362096,-1.345158,0.822402,0.833629,...,-1.167983,1.001054,0.665085,-1.254582,0.853575,0.869083,-1.485269,-1.129686,0.924417,InChI=1S/C10H11N3O3S/c1-7-6-10(12-16-7)13-17(1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0.856109,0.956310,0.661301,0.935187,-1.688016,0.720890,0.792188,0.781404,0.822402,0.833629,...,0.966776,-1.034894,-1.254508,-1.254582,0.853575,-1.166458,0.818542,-1.129686,-1.094787,InChI=1S/C9H7N7O2S/c1-15-4-14-7(16(17)18)9(15)...
654,-0.246768,-1.307527,-1.739882,-0.790100,0.701175,0.720890,0.792188,0.781404,0.822402,0.833629,...,0.966776,1.001054,0.956249,0.812115,0.853575,0.869083,-1.485199,0.893099,0.924417,InChI=1S/C9H9Cl2NO/c1-2-9(13)12-6-3-4-7(10)8(1...
655,-1.382356,-0.491157,-1.391809,0.190800,-0.826699,-1.453716,-0.894064,-1.339409,0.380184,-1.699483,...,-1.058731,1.001054,-0.672675,0.812115,0.853575,0.869083,0.818542,0.893099,0.924417,"InChI=1S/C9H9N3O2S2/c10-7-1-3-8(4-2-7)16(13,14..."
656,-1.222589,0.167286,-2.129608,-0.144170,0.701175,-1.454018,-1.218476,-1.340825,0.822402,0.833629,...,0.966776,1.001054,0.956249,0.812115,0.853575,0.869083,0.818542,0.893099,0.924417,InChI=1S/C9H9N3S/c10-12-9-11-8(6-13-9)7-4-2-1-...


In [3]:
CP_features_list = CP.iloc[:, :-1].columns.to_list()
len(CP_features_list)

398

In [4]:
labels = pd.read_csv('data/Endpoints_9_Mitotox_biological_processes_658.csv')
labels_list= labels.iloc[:, 1:].columns.to_list()
labels_list

['apoptosis up',
 'cytotoxicity BLA',
 'cytotoxicity SRB',
 'ER stress',
 'heat shock',
 'microtubule up',
 'mitochondrial disruption up',
 'oxidative stress up',
 'proliferation decrease']

In [5]:
df = pd.merge(CP, labels, on='InChICode_standardised')
df

,viability_apoptosis_vb_percent_dead_only_Chromatin Modifiers,viability_apoptosis_vb_percent_dead_only_ER Stress/UPR,viability_apoptosis_vb_percent_dead_only_Apoptosis,viability_apoptosis_vb_percent_dead_only_RTK,viability_apoptosis_vb_percent_dead_only_Cytoskeletal Re-org/Integrin,viability_apoptosis_vb_percent_dead_only_Spindle Dynamics,viability_apoptosis_vb_percent_dead_only_Known cancer gene,viability_apoptosis_vb_percent_dead_only_RNA Polymerase II,viability_death_vb_num_live_cells_Metabolism,viability_death_vb_num_live_cells_WNT,...,InChICode_standardised,apoptosis up,cytotoxicity BLA,cytotoxicity SRB,ER stress,heat shock,microtubule up,mitochondrial disruption up,oxidative stress up,proliferation decrease
0,0.856109,0.956310,0.661301,0.935187,-1.688069,-1.454033,-1.362097,-1.345309,0.822402,0.833629,...,"InChI=1S/C32H41NO2/c1-31(2,3)26-18-16-25(17-19...",1,1,0,1,0,0,0,0,0
1,-1.728855,0.956310,-2.264476,0.194311,-1.688069,0.720890,0.792188,0.781404,-1.410468,0.833629,...,InChI=1S/C10H10N2O/c1-8-7-10(13)12(11-8)9-5-3-...,0,0,0,0,0,0,0,0,0
2,0.604550,-1.535441,-0.053351,-1.656308,0.701175,0.720890,0.792188,0.781404,0.822402,0.833629,...,"InChI=1S/C10H10N4O2S/c11-8-2-4-9(5-3-8)17(15,1...",0,0,0,0,0,0,0,0,0
3,0.856109,-1.535441,0.661301,0.935187,0.701175,0.720890,0.792188,0.781404,-0.984302,-0.756872,...,"InChI=1S/C10H11ClO3/c1-10(2,9(12)13)14-8-5-3-7...",0,0,0,0,0,0,0,0,0
4,-0.495514,-0.138102,-1.731964,-0.691095,-1.688069,-1.454033,-1.362096,-1.345158,0.822402,0.833629,...,InChI=1S/C10H11N3O3S/c1-7-6-10(12-16-7)13-17(1...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0.856109,0.956310,0.661301,0.935187,-1.688016,0.720890,0.792188,0.781404,0.822402,0.833629,...,InChI=1S/C9H7N7O2S/c1-15-4-14-7(16(17)18)9(15)...,1,0,1,0,0,0,1,1,1
654,-0.246768,-1.307527,-1.739882,-0.790100,0.701175,0.720890,0.792188,0.781404,0.822402,0.833629,...,InChI=1S/C9H9Cl2NO/c1-2-9(13)12-6-3-4-7(10)8(1...,1,1,1,1,0,0,1,1,1
655,-1.382356,-0.491157,-1.391809,0.190800,-0.826699,-1.453716,-0.894064,-1.339409,0.380184,-1.699483,...,"InChI=1S/C9H9N3O2S2/c10-7-1-3-8(4-2-7)16(13,14...",0,0,0,0,0,0,0,1,0
656,-1.222589,0.167286,-2.129608,-0.144170,0.701175,-1.454018,-1.218476,-1.340825,0.822402,0.833629,...,InChI=1S/C9H9N3S/c10-12-9-11-8(6-13-9)7-4-2-1-...,1,1,0,0,1,0,0,0,0


In [6]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from numpy import argmax
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict

In [7]:
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

In [8]:
labels_list[-1:]

['proliferation decrease']

In [9]:
list_of_list = []

for labels_name in labels_list[:]:
    print(labels_name)
    
    df_assay = df.dropna(subset=[labels_name]).reset_index(drop=True)
    print(df_assay[labels_name].value_counts())
    
    count = 0
    for i in range(42,47):
        
        skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)
        
        for train_index_outer, test_index in skf.split(df_assay, df_assay[labels_name]):
        
            assay_results = pd.DataFrame()
            val_results = pd.DataFrame()

            count += 1
            save_name = f"{labels_name}_{count}"
            print(f"\n ***{save_name}***\n")

            train, test = df_assay.iloc[train_index_outer], df_assay.iloc[test_index]

            print("len(Train)", len(train))
            print("len(Test)", len(test))

            X_train= train[CP_features_list].to_numpy()
            X_test= test[CP_features_list].to_numpy()

            y_train= train[labels_name].to_numpy()
            y_test= test[labels_name].to_numpy()

            print("shape(X_Train)", (X_train.shape))
            print("shape(y_Train)", (y_train.shape))

            #break

            rf = RandomForestClassifier(n_jobs=20, random_state=42)


            # Instantiate the RandomHalving search model
            param_dist_grid = { 
            'max_depth': randint(5, 20),
            'max_features': randint(20, 150),
            'min_samples_leaf': randint(5, 15),
            'min_samples_split': randint(5, 15),
            'n_estimators':[200, 400, 600],
            'bootstrap': [True, False],
            'oob_score': [False],
            'random_state': [42],
            'criterion': ['gini', 'entropy'],
            'n_jobs': [20],
            'class_weight' : [None, 'balanced']
                }

            inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)    
            rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid,
                                        factor=2, random_state=i, n_jobs=20, verbose=1, cv = inner_cv)



            rsh.fit(X_train, y_train)
            classifier = rsh.best_estimator_
            classifier.fit(X_train, y_train)

            test_proba = classifier.predict_proba(X_test)[:,1]
            test_pred = to_labels(test_proba, 0.50)
            ba_old=balanced_accuracy_score(y_test, test_pred) 
            print("Old BA", ba_old.round(2))

            fpr, tpr, thresholds = metrics.roc_curve(y_test, test_proba)
            AUC = metrics.auc(fpr, tpr)
            print("AUC", AUC.round(2))

            cross_val_prob = cross_val_predict(classifier, X_train, y_train, cv=inner_cv, method='predict_proba')[:, 1]
            fpr, tpr, thresholds = metrics.roc_curve(y_train, cross_val_prob)
                        # get the best threshold
            J = tpr - fpr
            ix = argmax(J)
            best_thresh = thresholds[ix]
            print('Best Threshold=%f' % (best_thresh))

            test_preds_labels = to_labels(test_proba, best_thresh)
            ba_new=balanced_accuracy_score(y_test, test_preds_labels)
            print("New BA", ba_new.round(2))
            print(" ")

            row= [labels_name, save_name, "Ontology_pvalue", AUC, ba_new]
            list_of_list.append(row)

            metrics_df_CP= pd.DataFrame(list_of_list, columns=["labels_name", "epoch", "Features", "AUC", "ba"])
            #metrics_df_CP.to_csv("BAK_metrics_df_Ontology_pvalue_multiple_runs.csv", index=False)



apoptosis up
0    502
1    156
Name: apoptosis up, dtype: int64

 ***apoptosis up_1***

len(Train) 493
len(Test) 165
shape(X_Train) (493, 398)
shape(y_Train) (493,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 493
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.62
AUC 0.67
Best Threshold=0.348291
New BA 0.62
 

 ***apoptosis up_2***

len(Train) 493
len(Test) 165
shape(X_Train) (

----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.62
AUC 0.65
Best Threshold=0.357960
New BA 0.63
 

 ***apoptosis up_11***

len(Train) 494
len(Test) 164
shape(X_Train) (494, 398)
shape(y_Train) (494,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 494
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2


Old BA 0.6
AUC 0.67
Best Threshold=0.381801
New BA 0.66
 

 ***apoptosis up_20***

len(Train) 494
len(Test) 164
shape(X_Train) (494, 398)
shape(y_Train) (494,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 494
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.64
AUC 0.66
Best Threshold=0.413106
New BA 0.65
 
cytotoxicity BLA
0    526
1    132
Name: cytotoxicity BLA, dtype: int64

 

----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.54
AUC 0.68
Best Threshold=0.251301
New BA 0.61
 

 ***cytotoxicity BLA_10***

len(Train) 493
len(Test) 165
shape(X_Train) (493, 398)
shape(y_Train) (493,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 493
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter

Old BA 0.55
AUC 0.61
Best Threshold=0.318464
New BA 0.64
 

 ***cytotoxicity BLA_19***

len(Train) 494
len(Test) 164
shape(X_Train) (494, 398)
shape(y_Train) (494,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 494
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.53
AUC 0.68
Best Threshold=0.280924
New BA 0.62
 

 ***cytotoxicity BLA_20***

len(Train) 494
len(Test) 164
shape(X_Tra

...

----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.5
AUC 0.58
Best Threshold=0.095549
New BA 0.53
 

 ***mitochondrial disruption up_14***

len(Train) 493
len(Test) 165
shape(X_Train) (493, 398)
shape(y_Train) (493,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 493
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA

----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.49
AUC 0.45
Best Threshold=0.213652
New BA 0.49
 

 ***oxidative stress up_4***

len(Train) 494
len(Test) 164
shape(X_Train) (494, 398)
shape(y_Train) (494,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 494
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
it

----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.5
AUC 0.59
Best Threshold=0.112094
New BA 0.57
 

 ***oxidative stress up_13***

len(Train) 493
len(Test) 165
shape(X_Train) (493, 398)
shape(y_Train) (493,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 493
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.5
AUC

----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.5
AUC 0.58
Best Threshold=0.119298
New BA 0.59
 

 ***proliferation decrease_3***

len(Train) 494
len(Test) 164
shape(X_Train) (494, 398)
shape(y_Train) (494,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 494
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------


----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.5
AUC 0.57
Best Threshold=0.143319
New BA 0.55
 

 ***proliferation decrease_12***

len(Train) 494
len(Test) 164
shape(X_Train) (494, 398)
shape(y_Train) (494,)
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 20
max_resources_: 494
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 24
n_resources: 20
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 1
n_candidates: 12
n_resources: 40
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 6
n_resources: 80
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 3
n_resources: 160
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 2
n_resources: 320
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Old BA 0.5


In [10]:
metrics_df= pd.DataFrame(list_of_list, columns=["labels_name", "Features", "AUC", 
                                               "ba"])
metrics_df

ValueError: 4 columns passed, passed data had 5 columns

In [ ]:
importances = classifier.feature_importances_
importances

In [ ]:
filtered_CP_imp_features_list = np.array(CP_features_list)[np.array(importances>(importances.mean() + 1.5*importances.std()))]
len(set(filtered_CP_imp_features_list))

In [ ]:
filtered_CP_imp_features_list